In [99]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime

df = pd.read_csv("full_clean_raw.csv")

df['return'] = df['total_pymnt']/df['funded_amnt'] #% return for each loan
df['profit'] = df['total_pymnt'] - df['funded_amnt'] #calculate profit per loan

list(df.columns.values)

['Unnamed: 0',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'is_inc_v',
 'issue_d',
 'pymnt_plan',
 'purpose',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'not_compliant',
 'status',
 'inactive_loans',
 'bad_loans',
 'emp_length_num',
 'pub_rec_zero',
 'short_emp',
 'payment_inc_ratio',
 'final_d',
 'last_record_none',
 'last_major_derog_none',
 'home_ownership',
 'annual_inc',
 'addr_state',
 'dti',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'open_acc',
 'pub_rec',
 'total_acc',
 'inc_v_binary',
 'deliquency',
 'status_binary',
 'return',
 'profit']

In [ ]:
#correlations
df.corr() #correlation matrix

In [ ]:
#return % by loan status
pd.pivot_table(df, index="status", values="return") 
df_pre_agg = df.groupby('status').agg({'total_pymnt' : 'sum', 'funded_amnt' : 'sum'})
df_pre_agg['return'] = df_pre_agg['total_pymnt']/df_pre_agg['funded_amnt']
df_pre_agg['return']

In [ ]:
#return by employement status
pd.pivot_table(df, index="emp_length_num", values="return") #not really anything interesting

In [ ]:
#return by grades
pd.pivot_table(df, index="sub_grade", values="return", aggfunc=len)

In [ ]:
#return positive in each category
df['pos_return'] = df['return'].apply(lambda x: 1 if x>=1 else 0)
pd.pivot_table(df, index="sub_grade", values="pos_return")

In [ ]:
#profit by status
df_prof = df.groupby('status').agg({'profit' : 'sum', 'funded_amnt' : 'sum'})
df_prof['margin'] = df_prof['profit']/df_prof['funded_amnt']
sum(df_prof['profit'])/sum(df_prof['funded_amnt']) #total profit margin

In [ ]:
#profit by year
df['year'] = df['issue_d'].apply(lambda x: x[:4])
df_y = df.groupby('year').agg({'profit' : 'sum', 'funded_amnt' : 'sum', 'member_id': 'count'})
df_y['margin'] = df_y['profit']/df_y['funded_amnt']
df_y #profit margin is between -4% and 10% before 2012, but is -33% and -74% in 2013 and 2014 respectively


analysis = pd.pivot_table(df, values='member_id', index='year', columns='status', aggfunc=len, fill_value=0)
for col in analysis.columns:
    analysis[col] = (analysis[col]/df_y['member_id'] * 100).round()
analysis